In [3]:
#come up with a Brochure of a company

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [4]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [7]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent and scrape links from a url 
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [8]:
nk = Website("https://nike.com")
# nk.links

# '#skip-to-content',
#  'https://www.nike.com/jordan',
#  'https://www.nike.com/w/converse-akmjx',
#  'https://www.nike.com/retail',
#  'https://www.nike.com/help',
#  'https://www.nike.com/help',
#  'https://www.nike.com/orders/details/',
#  'https://www.nike.com/help/a/shipping-delivery',
#  'https://www.nike.com/help/a/returns-policy',
#  'https://www.nike.com/help/a/change-cancel-order',
#  'https://www.nike.com/help/a/size-charts',
#  'https://www.nike.com/help/#contact',
#  'https://www.nike.com/membership',
#  'https://www.nike.com/promo-code',
#  'https://www.nike.com/product-advice',
#  '#site-feedback',
#  'https://www.nike.com/membership',
#  'https://www.nike.com/register',

In [9]:
# find relavent links using one shot prompting, respond with json

link_system_prompt = "provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages, or best products page.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"},
        {"type": "Membership page": "url": "https://another.full.url/Membership"}
    ]
}
"""

In [10]:
print(link_system_prompt)


provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages, or best products page.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"},
        {"type": "Membership page": "url": "https://another.full.url/Membership"}
    ]
}



In [11]:
#list out all links(relavent)
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [13]:
# print(get_links_user_prompt(nk))


#output
# Here is the list of links on the website of https://nike.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
# Links (some might be relative links):
# #skip-to-content
# https://www.nike.com/jordan
# https://www.nike.com/w/converse-akmjx
# https://www.nike.com/retail
# https://www.nike.com/help
# https://www.nike.com/help
# https://www.nike.com/orders/details/
# https://www.nike.com/help/a/shipping-delivery
# https://www.nike.com/help/a/returns-policy
# https://www.nike.com/help/a/change-cancel-order
# https://www.nike.com/help/a/size-charts
# https://www.nike.com/help/#contact
# https://www.nike.com/membership

In [14]:
#use openai model call to get results in json

def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [15]:
# nike = Website("https://nike.com")
# nike.links



# output
# ['#skip-to-content',
#  'https://www.nike.com/jordan',
#  'https://www.nike.com/w/converse-akmjx',
#  'https://www.nike.com/retail',
#  'https://www.nike.com/help',
#  'https://www.nike.com/help',
#  'https://www.nike.com/orders/details/',
#  'https://www.nike.com/help/a/shipping-delivery',
#  'https://www.nike.com/help/a/returns-policy',
#  'https://www.nike.com/help/a/change-cancel-order',
#  'https://www.nike.com/help/a/size-charts',
#  'https://www.nike.com/help/#contact',
#  'https://www.nike.com/membership',
#  'https://www.nike.com/promo-code',

In [16]:
get_links("https://nike.com")

{'links': [{'type': 'about page', 'url': 'https://about.nike.com/en'},
  {'type': 'careers page', 'url': 'https://careers.nike.com/'},
  {'type': 'sustainability page',
   'url': 'https://www.nike.com/sustainability'},
  {'type': 'company news', 'url': 'http://news.nike.com/'}]}

In [56]:
# part 2 make brochure












In [17]:
# call out chatgpt mini and get relavent links

def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [18]:
# print(get_all_details("https://nike.com"))


#output
# Found links: {'links': [{'type': 'about page', 'url': 'https://about.nike.com/en'}, {'type': 'careers page', 'url': 'https://careers.nike.com/'}, {'type': 'company news', 'url': 'http://news.nike.com/'}, {'type': 'investor relations', 'url': 'http://investors.nike.com/'}, {'type': 'sustainability page', 'url': 'https://www.nike.com/sustainability'}, {'type': 'membership page', 'url': 'https://www.nike.com/membership'}, {'type': 'stories page', 'url': 'https://www.nike.com/stories'}]}
# Landing page:
# Webpage Title:
# Nike. Just Do It. Nike.com
# Webpage Contents:
# Skip to main content
# Find a Store
# Help
# Help
# Order Status
# Shipping & Delivery
# Returns
# Order Cancellation

In [19]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [20]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [21]:
get_brochure_user_prompt("Nike", "https://nike.com")


Found links: {'links': [{'type': 'about page', 'url': 'https://about.nike.com/en'}, {'type': 'careers page', 'url': 'https://careers.nike.com/'}, {'type': 'sustainability page', 'url': 'https://www.nike.com/sustainability'}, {'type': 'membership page', 'url': 'https://www.nike.com/membership'}, {'type': 'company news', 'url': 'http://news.nike.com/'}]}


"You are looking at a company called: Nike\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nNike. Just Do It. Nike.com\nWebpage Contents:\nSkip to main content\nFind a Store\nHelp\nHelp\nOrder Status\nShipping & Delivery\nReturns\nOrder Cancellation\nSize Charts\nContact Us\nMembership\nPromotions & Discounts\nProduct Advice\nSend Us Feedback\nJoin Us\nSign In\nNew\nFeatured\nNew Arrivals\nBest Sellers\nLatest Drops\nStyle Your Air\nVomero 18\nFairway Ready\nSNKRS Launch Calendar\nShop New\nMen\nWomen\nKids\nShop All\nTrending\n24.7 Collection\nNike Style By\nSo Win Collection\nACG Essentials\nRare Air Collection\nRunning Shoe Finder\nMen\nFeatured\nNew Arrivals\nBest Sellers\nLatest Drops\nStyle Your Air\nSpring Essentials\nShop All Sale\nShoes\nAll Shoes\nBasketball\nLifestyle\nJordan\nRetro Running\nRunning\nTraining & Gym\nSandals & Slides\nShoes $100 & Und

In [23]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("Nike", "https://nike.com")
